In [3]:
import pandas as pd
import numpy as np


In [5]:
df = pd.read_excel('Data/20181001-newindianexpress_sentence_classification_adjudicated_20181218.xlsx')

In [6]:
df

,url,sent_num,sentence,text,label
0,http://www.newindianexpress.com/states/andhra-...,1-1,Anantapur Lawyers to Revive Samaikya Agitation...,Anantapur Lawyers to Revive Samaikya Agitation...,2.0
1,http://www.newindianexpress.com/states/andhra-...,1-2,A meeting of lawyers was held in the town on S...,Anantapur Lawyers to Revive Samaikya Agitation...,0.0
2,http://www.newindianexpress.com/states/andhra-...,1-3,"Speaking to reporters later, advocates’ leader...",Anantapur Lawyers to Revive Samaikya Agitation...,0.0
3,http://www.newindianexpress.com/states/andhra-...,1-4,“Bifurcation is being done without taking into...,Anantapur Lawyers to Revive Samaikya Agitation...,0.0
4,http://www.newindianexpress.com/states/andhra-...,1-5,The Bill did not contain any proposals for dev...,Anantapur Lawyers to Revive Samaikya Agitation...,0.0
5,http://www.newindianexpress.com/states/andhra-...,1-6,"As part of their protests, the lawyers have de...",Anantapur Lawyers to Revive Samaikya Agitation...,2.0
6,http://www.newindianexpress.com/states/andhra-...,1-7,"“We appeal to all Samaikyandhra supporters, st...",Anantapur Lawyers to Revive Samaikya Agitation...,2.0
7,http://www.newindianexpress.com/states/andhra-...,1-8,"On January 28, the lawyers decided to take out...",Anantapur Lawyers to Revive Samaikya Agitation...,2.0
8,http://www.newindianexpress.com/states/andhra-...,1-9,State bar Association vice-president A Rami Re...,Anantapur Lawyers to Revive Samaikya Agitation...,2.0
9,http://www.newindianexpress.com/states/andhra-...,1-10,The lawyers body will also organise a meeting ...,Anantapur Lawyers to Revive Samaikya Agitation...,0.0


In [8]:
df['sentence']

'Anantapur Lawyers to Revive Samaikya Agitation, to Lay Siege to Courts Today 27th January 2014 09:58 AM Renewing the Samikyandhra agitation, the lawyers of the district have decided to take on a series of protests starting with laying siege to courts on Monday.'